# Initialization

In [23]:
import pandas as pd
import re
import numpy as np
import urllib
from bs4 import BeautifulSoup as Soup
import os

In [195]:
speakers = pd.DataFrame(columns=["speaker_id", 
                                "first_name", 
                                "last_name", 
                                "type",
                                "party", 
                                "state", 
                                "district",
                                "bio_guide_id",
                                "congress_id"])

In [196]:
speakers

,speaker_id,first_name,last_name,type,party,state,district,bio_guide_id,congress_id


# Master Mods Parsing

In [197]:
def get_all_extensions(file):
    handler = open(file).read()
    soup = Soup(handler, "lxml")
    return soup.find_all("extension")

In [198]:
# extensions = get_all_extensions("mastermods.xml")

In [199]:
def get_cong_member_tag(cong_member_extension):
    cong_member_tag = cong_member_extension.find("congmember")
    return cong_member_tag

In [200]:
def get_party(cong_member_tag):
    try:
        return cong_member_tag.attrs['party']
    except:
        print("no party")
        print(cong_member_tag)
def get_type(cong_member_tag):
    try:
        return cong_member_tag.attrs['type']
    except:
        print("no type")
        print(cong_member_tag)
def get_authority_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['authorityid']
    except:
        print("no authority_id")
        print(cong_member_tag)
def get_bioguide_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['bioguideid']
    except:
        print("no bioguide_id")
        print(cong_member_tag)
def get_state(cong_member_tag):
    try:
        return cong_member_tag.attrs['state']
    except:
        print("no state")
        print(cong_member_tag)
def get_congress_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['congress']
    except:
        print("no congress_id")
        print(cong_member_tag)

In [201]:
def get_district(cong_member_extension):
    district_tag = cong_member_extension.find("district")
    if district_tag == None:
        return None
    return district_tag.string

In [202]:
def get_first_name(cong_member_tag):
    name_tag = cong_member_tag.select("name[type='authority-fnf']")[0]
    if name_tag == []:
        print("no first_name")
        print(cong_member_tag)
        return None
    first_name = name_tag.text.split()[0]
    return first_name

In [203]:
def get_last_name(cong_member_tag):
    name_tag = cong_member_tag.select("name[type='authority-lnf']")[0]
    if name_tag == []:
        print("no first_name")
        print(cong_member_tag)
        return None
    full_name = name_tag.string
    return re.match("[^,]*", full_name).group(0)

In [204]:
filenames = os.listdir("/Users/halliday/projects/searchlight/parsing/mastermods")
filenames.remove('.DS_Store')
for filename in filenames:
    extensions = get_all_extensions("/Users/halliday/projects/searchlight/parsing/mastermods/" + filename)
    print("            " + filename)
    for extension in extensions:
        cong_member_tag = get_cong_member_tag(extension)
        if cong_member_tag == None:
            continue
        counter += 1
#         if filename == 'mods112.xml':
#             print(cong_member_tag)
        authority_id = get_authority_id(cong_member_tag)
        if speakers[speakers['speaker_id'] == authority_id]['speaker_id'].count() > 0 or authority_id == "" or authority_id == None:
            continue
        cong_member_dict = {'speaker_id': authority_id, 
                            'first_name': get_first_name(cong_member_tag),
                            'last_name': get_last_name(cong_member_tag),
                            'type': get_type(cong_member_tag),
                            'party': get_party(cong_member_tag),
                            'state': get_state(cong_member_tag),
                            'district': get_district(extension),
                            'bio_guide_id': get_bioguide_id(cong_member_tag),
                            'congress_id': get_congress_id(cong_member_tag)}
        speakers = speakers.append(cong_member_dict, ignore_index=True)

            mods113.xml
            mods107.xml
            mods106.xml
            mods112.xml
no authority_id
<congmember chamber="H" congress="112" state="NJ" type="REPRESENTATIVE">
<name type="parsed">DONALD M. PAYNE</name>
</congmember>
            mods110.xml
            mods111.xml
            mods105.xml
            mods114.xml
            mods108.xml
            mods109.xml


In [216]:
speakers = speakers.sort_values('last_name')

In [219]:
speakers.to_csv('masterspeakers.csv', index=False)

# Speeches Parsing + Local Mods Connection